<a href="https://colab.research.google.com/github/youngmoo/ECES-434/blob/main/Class%206.1%20(2021-02-15).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **ECES-434: Class 6.1 (2021-02-15)**
Week 6: Let's start focusing on the final project!

In [ ]:
import numpy as np                      # NumPy, abbreviated as np
import matplotlib.pyplot as plt         # MatplotLib PyPlot module, abbreviated as plt
from matplotlib import animation, rc    # MatplotLib animation module
%matplotlib inline
from scipy import signal                # SciPy's signal module, for DSP functions
import soundfile as sf                  # Switching to the soundfile module for reading and writing soundfiles

import IPython.display as ipd           # Interactive Python display module, for playing sounds
from IPython.display import HTML        # For displaying animations
rc('animation', html='jshtml')          # Provides animation controls

ClassPath = '/content/drive/My Drive/ECES-434 Sessions/Class 6-1/'

In [ ]:
# CHANGE THIS to your Drexel username!!
username = 'anonymous'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Custom plotting functions
Because we're always plotting...

## plotSpectrogram

In [ ]:
def plotSpectrogram(sig, fs, win='hann', nseg=512, olap=256, fft_len=512):
  f1, t1, Sxx = signal.spectrogram(sig, fs, window=win, nperseg=nseg, noverlap=olap, nfft=fft_len)

  fig = plt.figure(figsize=(16,6))

  plt.pcolormesh(t1, f1, 20*np.log10(np.abs(Sxx)))
  plt.ylabel('Frequency (Hz)')
  plt.xlabel('Time (sec)')
  return fig, plt

## myPlot(): properly formats time domain plot of a signal

In [ ]:
def myPlot(sig, fs=44100):
  fig = plt.figure(figsize=(16,4))
  t = np.arange(len(sig)) / fs
  plt.plot(t, sig)
  plt.xlabel('Time (sec)')
  return fig, plt

## myPlotFFT(): properly formats frequency domain plot of a signal

In [ ]:
def myPlotFFT(sig, n_fft=0, x_lim=22050, fs=44100):
  if n_fft==0:                 
    n_fft = len(sig)                    # Default to length of input signal
  S = np.fft.fft(sig, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  fig = plt.figure(figsize=(16,4))
  plt.plot(f, 20*np.log10(np.abs(S)))
  plt.xlim(0, x_lim)
  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Magnitude (dB)')
  return fig, plt  

## myPlotFFTPhase

In [ ]:
def myPlotFFTPhase(sig, n_fft=0, x_lim=22050, fs=44100):
  if n_fft==0:                 
    n_fft = len(sig)                    # Default to length of input signal
  S = np.fft.fft(sig, n_fft)
  N = len(S)
  f = np.arange(N) * fs / N
  fig = plt.figure(figsize=(16,4))
  plt.plot(f, np.unwrap(np.angle(S)))
  plt.xlim(0, x_lim)
  plt.xlabel('Frequency (Hz)')
  plt.ylabel('Phase (radians)')
  return fig, plt

## Custom FFT animation functions

In [ ]:
n_o = 0
f_size = 2048
n_hop = f_size / 2
N_fft = 4096
fs = 44100
f = np.arange(N_fft) * fs / N_fft

# First set up the figure, the axis, and the plot element we want to animate
def setupAnimFFT(x_lim=(0,20000), y_lim=(-120,100)):
  fig = plt.figure(figsize=(14,6))
  ax = plt.axes(xlim=x_lim,ylim=y_lim)
  plt.close()   # Don't output the final figure separately
  line, = ax.plot([], [])
  return fig, line

# initialization function: plot the background of each frame
def initAnimFFT():
    line.set_data([], [])
    return (line,)

# animation function. This is called sequentially  
def animateFFT(i, sig):
    n1 = int(n_o + n_hop*i)
    n2 = int(n_o + n_hop*i + f_size)

    x_i = sig[n1:n2]
    X_i = np.fft.fft(x_i * np.hanning(len(x_i)), n=N_fft)
    X_mag = 20*np.log(np.abs(X_i))

    line.set_data(f, X_mag)
    return (line,)  

# Usage:
# fig, line = setupAnimFFT()
# anim = animation.FuncAnimation(fig, animateFFT, init_func=initAnimFFT, frames=120, fargs=(signal,), interval=1000/30, blit=True)
# anim

# Today's sound file

In [ ]:
aha_s, fs44 = sf.read(ClassPath + 'TakeOnMe-44kHz.wav')
aha = np.mean(aha_s,axis=1)
ipd.Audio(aha,rate=fs44)

# Inverse DFT and STFT


In [ ]:
s = aha[1:2048]
S = np.fft.fft(s)
s_inv = np.fft.ifft(S)
fig = plt.figure(figsize=(16,8))
plt.subplot(311)
plt.plot(s)
plt.subplot(312)
plt.plot(s_inv)
plt.subplot(313)
plt.plot(s - np.real(s_inv))

## An "inverse STFT"

In [ ]:
f_size = 2048
n_hop = f_size / 2
N_fft = f_size
fs = 44100

#L = 128
#H_id = np.append(np.ones(L), np.zeros(int(f_size/2)-L))
#H_id = np.append(H_id, np.flipud(H_id))

H_id = np.ones(f_size)

def fftFilter(sig, n_frames=0):
  out = np.zeros(len(sig))

  if n_frames == 0:
    n_frames = int(len(sig)/n_hop) - 1
    print(n_frames)

  for i in range(n_frames):
    n1 = int(n_hop*i)
    n2 = int(n_hop*i + f_size)

    x_i = sig[n1:n2]
    X_i = np.fft.fft(x_i * np.hanning(len(x_i)), n=N_fft)
    X_f = X_i * H_id
    x_f = np.fft.ifft((X_f))

    out[n1:n2] += np.real(x_f)

  return out

In [ ]:
aha2 = fftFilter(aha)
myPlot(aha2)
ipd.Audio(aha2,rate=fs)

## A closer look at phase issues

In [ ]:
f_start = 40000
f_size = 2048
nFFT = 2048
L = 512

S = np.fft.fft(aha[f_start:f_start+f_size] * np.hanning(f_size), nFFT)
H = np.append(np.ones(L), np.zeros(int(nFFT/2)-L))
H = np.append(H, np.flipud(H))

S_f = S # * H
s_f = np.fft.ifft((S_f))
fig = plt.figure(figsize=(16,6))
plt.subplot(211)
plt.plot(aha[f_start:f_start+f_size] * np.hanning(f_size))
plt.subplot(212)
plt.plot(s_f)

## What's happening (ideal LPF)?



In [ ]:
fs = 44100
N = 1024
L = 64
H_ideal = np.append(np.ones(L),np.zeros(N-L))

# Filter needs to be symmetric
H_ideal_sym = np.append(H_ideal, np.flipud(H_ideal))

fig = plt.figure(figsize=(16,4))
f = np.arange(len(H_ideal_sym)) * fs / len(H_ideal_sym)
plt.plot(f, H_ideal_sym)

In [ ]:
f_Hz = np.arange(-N,N)*(fs/2) / N
#f_norm = np.arange(-N,N) / N
#f_rad = np.arange(-N,N)*np.pi / N

fig = plt.figure(figsize=(16,4))
plt.plot(f_Hz, np.fft.fftshift(H_ideal_sym))

plt.xlabel('Frequency (Hz)')
#plt.xlabel('Frequency (normalized)')
#plt.xlabel('Frequency (radians)')

In [ ]:
h_ideal_sym = np.fft.ifft(H_ideal_sym)
h_ideal_shift = np.fft.fftshift(np.real(h_ideal_sym))

n = np.arange(-N,N)
fig = plt.figure(figsize=(16,4))
plt.plot(n, h_ideal_shift)
plt.xlabel('Samples')

## Another "inverse STFT"

In [ ]:
#L = 128
#H_id = np.append(np.ones(L), np.zeros(int(f_size/2)-L))
#H_id = np.append(H_id, np.flipud(H_id))
H_id = np.ones(f_size)

def fftFilter2(sig, n_frames=0):
  out = np.zeros(len(sig))

  if n_frames == 0:
    n_frames = int(len(sig)/n_hop) - 1

  for i in range(n_frames):
    n1 = int(n_hop*i)
    n2 = int(n_hop*i + f_size)

    x_i = sig[n1:n2]
    X_i = np.fft.fft(x_i * np.hanning(len(x_i)), n=N_fft)
    X_f = X_i * H_id
    x_f = np.fft.ifft(np.real(X_f))

    out[n1:n2] += np.real(x_f * np.hanning(f_size))

  return out

In [ ]:
aha2 = fftFilter2(aha)
myPlot(aha2)
ipd.Audio(aha2,rate=fs)

# Final Project: Compression

In [ ]:
len(aha)

## How much space does it take?

In [ ]:
len(aha) * ...

In [ ]:
n_bits = 16
aha_q = np.floor(aha * 2**(n_bits-1))
myPlot(aha_q)
ipd.Audio(aha_q, rate=fs44)

# Perceptual audio coding

In [ ]:
fs = 22050
f0 = 100
t = np.arange(fs) / fs
s = np.sin(2*np.pi*f0*t)
ipd.Audio(s,rate=fs)

In [ ]:
f_delta = 5

s_mod = np.sin(2*np.pi*f0*t + 5*np.sin(2*np.pi*f_delta*t))
ipd.Audio(s_mod,rate=fs)